In [ ]:
!pip install catboost
!pip install feature_engine

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, ElasticNet

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.simplefilter('ignore')

In [ ]:
ss = pd.read_csv('Sample Submission.csv')
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [ ]:
train.head(3)
test.head(3)
ss.head(3)

InvoiceNo  StockCode  ...  Ordered_Revenue  Cancelled_Revenue
0       6141       1583  ...            11.25                0.0
1       6349       1300  ...            11.70                0.0
2      16783       2178  ...            23.80                0.0

[3 rows x 12 columns]

InvoiceNo  StockCode  Description  ...  Country Ordered  Cancelled
0       3139       1709         1079  ...       35       3          0
1        617        510         3457  ...       35       1          0
2      14653        604          694  ...       35      36          0

[3 rows x 9 columns]

UnitPrice
0        100
1        100
2        100

In [ ]:
ID_COL ,TARGET_COL = 'InvoiceNo','UnitPrice'

num_cols = ['Quantity','Ordered','Cancelled','Ordered_Revenue','Cancelled_Revenue']
cat_cols = ['Country','StockCode','Description','CustomerID']
date_cols = ['InvoiceDate']

In [ ]:
def download_preds(preds_test, file_name = 'hacklive_sub.csv'):

  ## 1. Setting the target column with our obtained predictions
  ss[TARGET_COL] = preds_test

  ## 2. Saving our predictions to a csv file

  ss.to_csv(file_name, index = False)

  ## 3. Downloading and submitting the csv file
  from google.colab import files
  files.download(file_name)

In [ ]:
train.shape, test.shape

((284780, 12), (122049, 9))

In [ ]:
df = pd.concat([train, test], axis=0).reset_index(drop = True)
df.shape

(406829, 12)

In [ ]:
df['Country'] = df['Country'].astype(str)
df['StockCode'] = df['StockCode'].astype(str)
df['Description'] = df['Description'].astype(str)
df['CustomerID'] = df['CustomerID'].astype(str)
# df[cat_cols] = df[cat_cols].apply(lambda x: pd.factorize(x)[0])

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_fill = imputer.fit_transform(df[['Cancelled','Cancelled_Revenue']])
df_fill

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [ ]:
df_fill_d = pd.DataFrame(df_fill,columns=['Cancelled','Cancelled_Revenue'])
df_fill_d.to_csv('CancelledRevenue.csv',index=False)

In [ ]:
rto = pd.read_csv('OrderedRevenue.csv')
rto['InvoiceNo'] = df['InvoiceNo']
df.loc[df.InvoiceNo.isin(rto.InvoiceNo), ['Ordered_Revenue']] = rto.loc[rto.InvoiceNo.isin(df.InvoiceNo),['Ordered_Revenue']].values

In [ ]:
rto1 = pd.read_csv('CancelledRevenue.csv')
rto1['InvoiceNo'] = df['InvoiceNo']
df.loc[df.InvoiceNo.isin(rto1.InvoiceNo), ['Cancelled_Revenue']] = rto1.loc[rto1.InvoiceNo.isin(df.InvoiceNo),['Cancelled_Revenue']].values

In [ ]:
df = df.fillna(0)
df.isnull().sum().sum()

0

In [ ]:
df.describe()

StockCode  ...  Description_min_Cancelled
count  406829.000000  ...              406829.000000
mean      912.817557  ...                   0.000388
std       744.295663  ...                   0.047759
min         0.000000  ...                   0.000000
25%       289.000000  ...                   0.000000
50%       730.000000  ...                   0.000000
75%      1357.000000  ...                   0.000000
max      3683.000000  ...                  24.000000

[8 rows x 30 columns]

In [ ]:
df['UnitPrice'] = np.where(df['UnitPrice'] > 9000,1.25,df['UnitPrice'])

In [ ]:
df['Ordered'] = np.where(df['Ordered'] > 10000,12,df['Ordered'])

In [ ]:
df['Cancelled'] = np.where(df['Cancelled'] > 10000,2,df['Cancelled'])

In [ ]:
df['UnitPrice'] = np.where(df['UnitPrice'] > 9000,1.25,df['UnitPrice'])
df['Ordered'] = np.where(df['Ordered'] > 10000,12,df['Ordered'])
df['Cancelled'] = np.where(df['Cancelled'] > 10000,2,df['Cancelled'])

In [ ]:
df[cat_cols] = df[cat_cols].apply(lambda x: pd.factorize(x)[0])

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['InvoiceDate_day'] = df['InvoiceDate'].dt.day

df['InvoiceDate_dayofweek'] = df['InvoiceDate'].dt.dayofweek
df['InvoiceDate_year'] = df['InvoiceDate'].dt.year
df['InvoiceDate_month'] = df['InvoiceDate'].dt.month

In [ ]:
df['InvoiceDate_count'] = df['InvoiceDate'].map(df['InvoiceDate'].value_counts())
df['StockCode_count'] = df['StockCode'].map(df['StockCode'].value_counts())
df['Description_count'] = df['Description'].map(df['Description'].value_counts())



In [ ]:
df = df.fillna(0)
df.isnull().sum().sum()

In [ ]:
df = df.drop(columns=['InvoiceNo','Quantity'])

In [ ]:
df['Country_StockCode'] = df.groupby('StockCode')['Country'].transform('nunique')
df['Country_Description'] = df.groupby('Description')['Country'].transform('nunique')

In [ ]:
df['StockCode_mean_Ordered'] = df.groupby('StockCode')['Ordered'].transform('mean')
df['StockCode_max_Ordered'] = df.groupby('StockCode')['Ordered'].transform('max')
df['StockCode_min_Ordered'] = df.groupby('StockCode')['Ordered'].transform('min')

df['Description_mean_Ordered'] = df.groupby('Description')['Ordered'].transform('mean')
df['Description_max_Ordered'] = df.groupby('Description')['Ordered'].transform('max')
df['Description_min_Ordered'] = df.groupby('Description')['Ordered'].transform('min')

In [ ]:
df['StockCode_mean_Cancelled'] = df.groupby('StockCode')['Cancelled'].transform('mean')
df['StockCode_max_Cancelled'] = df.groupby('StockCode')['Cancelled'].transform('max')
df['StockCode_min_Cancelled'] = df.groupby('StockCode')['Cancelled'].transform('min')

df['Description_mean_Cancelled'] = df.groupby('Description')['Cancelled'].transform('mean')
df['Description_max_Cancelled'] = df.groupby('Description')['Cancelled'].transform('max')
df['Description_min_Cancelled'] = df.groupby('Description')['Cancelled'].transform('min')

In [ ]:
train_proc, test_proc = df[:train.shape[0]], df[train.shape[0]:].reset_index(drop = True)
features = [c for c in train_proc.columns if c not in [TARGET_COL,ID_COL]]
cat_num_cols = [c for c in features if c not in date_cols]

In [ ]:
trn, val = train_test_split(train_proc, test_size=0.2, random_state = 1)

###### Input to our model will be the features
X_trn, X_val = trn[features], val[features]

###### Output of our model will be the TARGET_COL
y_trn, y_val = trn[TARGET_COL], val[TARGET_COL]

##### Features for the test data that we will be predicting
X_test = test_proc[features]

In [ ]:
from sklearn.metrics import mean_squared_error

def av_metric(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
from sklearn.preprocessing import StandardScaler   
scaler = StandardScaler()
_ = scaler.fit(X_trn[cat_num_cols])

X_trn = scaler.transform(X_trn[cat_num_cols])
X_val = scaler.transform(X_val[cat_num_cols])
X_test = scaler.transform(X_test[cat_num_cols])

In [ ]:
############# Fitting and Predicting ################


clf = CatBoostRegressor(random_state=2)

 

_ = clf.fit(X_trn, y_trn)

### Instead of directly predicting the classes we will obtain the probability of positive class.
preds_val = clf.predict(X_val)
preds_test = clf.predict(X_test)

Learning rate set to 0.102578
0:	learn: 28.6443817	total: 63.4ms	remaining: 1m 3s
1:	learn: 28.1780992	total: 76.7ms	remaining: 38.3s
2:	learn: 27.6130619	total: 89ms	remaining: 29.6s
3:	learn: 27.1531751	total: 101ms	remaining: 25.3s
4:	learn: 26.6391701	total: 114ms	remaining: 22.6s
5:	learn: 26.2307786	total: 127ms	remaining: 21s
6:	learn: 25.8553141	total: 139ms	remaining: 19.8s
7:	learn: 25.4764174	total: 152ms	remaining: 18.8s
8:	learn: 25.1046980	total: 164ms	remaining: 18s
9:	learn: 24.7731180	total: 182ms	remaining: 18s
10:	learn: 24.4509511	total: 196ms	remaining: 17.6s
11:	learn: 24.1334084	total: 207ms	remaining: 17.1s
12:	learn: 23.8509167	total: 222ms	remaining: 16.9s
13:	learn: 23.4866616	total: 235ms	remaining: 16.5s
14:	learn: 23.2172894	total: 249ms	remaining: 16.3s
15:	learn: 22.9696186	total: 262ms	remaining: 16.1s
16:	learn: 22.7248843	total: 279ms	remaining: 16.1s
17:	learn: 22.4980718	total: 291ms	remaining: 15.9s
18:	learn: 22.2801978	total: 304ms	remaining: 15.

In [ ]:
av_metric_score = av_metric(y_val, preds_val)

print(f'AV metric score is: {av_metric_score}') # 163.09 - RF , 162.81 - LGBM , 165.27 - LR for country 

AV metric score is: 155.7158384841027


In [ ]:
# pd = np.expm1(preds_test)
# preds_test= np.clip(preds_test, train['UnitPrice'].min(), train['UnitPrice'].max())
download_preds(preds_test, file_name = '21cbg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# c,d - get_dummies, ID,SC- factorise , Robust Scaler - LGBM 163.011

In [ ]:
# day, month,year - LGBM 163.181

In [ ]:
# catnumLGBMd - 162.833  for country,description 
# Don't select DT 
# Don't run XGB


In [ ]:
def run_clf_kfold(clf, train, test, features):

  N_SPLITS = 5

  oofs = np.zeros(len(train))
  preds = np.zeros((len(test)))

  target = train[TARGET_COL]

  folds = StratifiedKFold(n_splits = N_SPLITS)
  stratified_target = pd.qcut(train[TARGET_COL], 10, labels = False, duplicates='drop')

  for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, stratified_target)):
    print(f'\n------------- Fold {fold_ + 1} -------------')

    ############# Get train, validation and test sets along with targets ################
  
    ### Training Set
    X_trn, y_trn = train[features].iloc[trn_idx], target.iloc[trn_idx]

    ### Validation Set
    X_val, y_val = train[features].iloc[val_idx], target.iloc[val_idx]

    ### Test Set
    X_test = test[features]

    ############# Scaling Data ################
    scaler = StandardScaler()
    _ = scaler.fit(X_trn)

    X_trn = scaler.transform(X_trn)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)


    ############# Fitting and Predicting ################

    _ = clf.fit(X_trn, y_trn)

    ### Instead of directly predicting the classes we will obtain the probability of positive class.
    preds_val = clf.predict(X_val)
    preds_test = clf.predict(X_test)

    fold_score = av_metric(y_val, preds_val)
    print(f'\nAV metric score for validation set is {fold_score}')

    oofs[val_idx] = preds_val
    preds += preds_test / N_SPLITS


  oofs_score = av_metric(target, oofs)
  print(f'\n\nAV metric for oofs is {oofs_score}')

  return oofs, preds

In [ ]:
clf = LGBMRegressor()
lgb_oofs, lgb_preds = run_clf_kfold(clf, train_proc, test_proc, cat_num_cols)

In [ ]:
# lgb_preds_t = np.expm1(lgb_preds)
download_preds(lgb_preds, file_name = '15th Nov.csv')

In [ ]:
!pip install feature_engine

In [ ]:
from sklearn.preprocessing import Normalizer

def run_gradient_boosting(clf, fit_params, train, test, features):
  N_SPLITS = 5
  oofs = np.zeros(len(train_proc))
  preds = np.zeros((len(test_proc)))

  target = train[TARGET_COL]

  folds = StratifiedKFold(n_splits = N_SPLITS)
  stratified_target = pd.qcut(train[TARGET_COL], 10, labels = False, duplicates='drop')

  feature_importances = pd.DataFrame()

  for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, stratified_target)):
    print(f'\n------------- Fold {fold_ + 1} -------------')

    ### Training Set
    X_trn, y_trn = train[features].iloc[trn_idx], target.iloc[trn_idx]

    ### Validation Set
    X_val, y_val = train[features].iloc[val_idx], target.iloc[val_idx]

    ### Test Set
    X_test = test[features]

    scaler = StandardScaler()
    _ = scaler.fit(X_trn)

    X_trn = scaler.transform(X_trn)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    _ = clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], **fit_params)

    fold_importance = pd.DataFrame({'fold': fold_ + 1, 'feature': features, 'importance': clf.feature_importances_})
    feature_importances = pd.concat([feature_importances, fold_importance], axis=0)

    ### Instead of directly predicting the classes we will obtain the probability of positive class.
    preds_val = clf.predict(X_val)
    preds_test = clf.predict(X_test)

    fold_score = av_metric(y_val, preds_val)
    print(f'\nAV metric score for validation set is {fold_score}')

    oofs[val_idx] = preds_val
    preds += preds_test / N_SPLITS


  oofs_score = av_metric(target, oofs)
  print(f'\n\nAV metric for oofs is {oofs_score}')

  feature_importances = feature_importances.reset_index(drop = True)
  fi = feature_importances.groupby('feature')['importance'].mean().sort_values(ascending = False)[:20][::-1]
  fi.plot(kind = 'barh', figsize=(12, 6))

  return oofs, preds, fi

In [ ]:
clf = LGBMRegressor(n_estimators = 1000,
                        learning_rate = 0.01,
                        colsample_bytree = 0.76,
                        metric = 'None',
                        )
fit_params = {'verbose': 100, 'early_stopping_rounds': 100, 'eval_metric': 'rmse'}

lgb_oofs, lgb_preds, fi = run_gradient_boosting(clf, fit_params, train_proc, test_proc, cat_num_cols)

In [ ]:
# lgb_preds_t = np.expm1(lgb_preds)
download_preds(lgb_preds, file_name = 'Af.csv')


In [ ]:
clf = LGBMRegressor(n_estimators = 5000,
                        learning_rate = 0.05,
                        colsample_bytree = 0.6,
                        metric = 'None',
                        )
fit_params = {'verbose': 300, 'early_stopping_rounds': 200, 'eval_metric': 'rmse'}

lgb_oofs, lgb_preds, fi = run_gradient_boosting(clf, fit_params, train_proc, test_proc, cat_num_cols)

In [ ]:
download_preds(lgb_preds, file_name = 'ETLGBM1.csv')

In [ ]:
clf = CatBoostRegressor(n_estimators = 5000,
                       learning_rate = 0.05,
                       rsm = 0.5, ## Analogous to colsample_bytree
                       random_state=2,
                       )

fit_params = {'verbose': 200, 'early_stopping_rounds': 200}

cb_oofs, cb_preds, fi = run_gradient_boosting(clf, fit_params, train_proc, test_proc, cat_num_cols)

In [ ]:
download_preds(cb_preds, file_name = '20cbrg.csv')

In [ ]:
clf = XGBRegressor(n_estimators = 5000,
                    max_depth = 6,
                    learning_rate = 0.05,
                    colsample_bytree = 0.5,
                    random_state=2,
                    )

fit_params = {'verbose': 200, 'early_stopping_rounds': 200}

xgb_oofs, xgb_preds, fi = run_gradient_boosting(clf, fit_params, train_proc, test_proc, cat_num_cols)

In [ ]:
download_preds(xgb_preds, file_name = '20xgb2.csv')

In [ ]:
import numpy as np
from keras.layers import Dense, Activation
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
train = train.drop(columns=['InvoiceDate','InvoiceNo','Quantity'])

In [ ]:
train['UnitPrice'] = np.where(train['UnitPrice'] > 9000,1.25,train['UnitPrice'])
train['Ordered'] = np.where(train['Ordered'] > 10000,12,train['Ordered'])
train['Cancelled'] = np.where(train['Cancelled'] > 10000,2,train['Cancelled'])

In [ ]:
train['Country'] = train['Country'].astype(str)
train['StockCode'] = train['StockCode'].astype(str)
train['Description'] = train['Description'].astype(str)
train['CustomerID'] = train['CustomerID'].astype(str)

cat_cols= ['Country','StockCode','Description','CustomerID']
train[cat_cols] = train[cat_cols].apply(lambda x: pd.factorize(x)[0])

In [ ]:
col =['Ordered','Cancelled','CustomerID','Country','StockCode','Description']
X = train[col]
Y = train['UnitPrice']

In [ ]:
Y = Y.apply(lambda x: np.log1p(x))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [ ]:
from feature_engine import variable_transformers as vt
yjt = vt.YeoJohnsonTransformer(variables = ['Quantity'])
_ = yjt.fit(X_train)

X_train = yjt.transform(X_train)
X_test = yjt.transform(X_test)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Initialising the ANN
model = Sequential()

# Adding the input layer and the first hidden layer
model.add(Dense(128, activation = 'relu', input_dim = 6))

# Adding the second hidden layer
model.add(Dense(units = 64, activation = 'relu'))

# Adding the third hidden layer
model.add(Dense(units = 32, activation = 'relu'))

# Adding the output layer

model.add(Dense(units = 1))

#model.add(Dense(1))
# Compiling the ANN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
# Fitting the ANN to the Training set
model.fit(X_train, y_train, batch_size = 5, epochs = 10)

In [ ]:
y_pred = model.predict(X_test)

plt.plot(y_test, color = 'red', label = 'Real data')
plt.plot(y_pred, color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()

In [ ]:
pred = pd.DataFrame(y_pred,columns=['UnitPrice']).to_csv('20NN.csv', index=False)

In [ ]:
c1 = ['StockCode','Description','Quantity','Country']
t1 = test[c1]
pred = model.predict(t1)

In [ ]:
pred = pd.DataFrame(pred,columns=['UnitPrice']).to_csv('NNpredlinear.csv', index=False)